## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import Callback

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [2]:
# Dropping the 'EIN' column, and swapping the drop column of 'NAME', using instead 'USE_CASE', 
# to improve performance of model
application_df.drop(["EIN", "USE_CASE"], axis=1, inplace=True)
application_df.head()


NAME APPLICATION_TYPE  \
0              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION  ORGANIZATION  STATUS     INCOME_AMT  \
0       Independent          C1000   Association       1              0   
1       Independent          C2000  Co-operative       1         1-9999   
2  CompanySponsored          C3000   Association       1              0   
3  CompanySponsored          C2000         Trust       1    10000-24999   
4       Independent          C1000         Trust       1  100000-499999   

  SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0                      N     5000              1  
1                      N   108590              1  
2                      N     5000              0  
3                      N     6692              1  
4                      N   142590              1

In [3]:
# Determine the number of unique values in each column.
uniqueValues = application_df.nunique()
print(uniqueValues)

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64


In [4]:
# Look at Name value counts for binning, since it appears to be likely to affect the model
name_count = application_df['NAME'].value_counts()
name_count.head()

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
Name: NAME, dtype: int64

In [5]:
# Cutoff value selected of 10, to focus on those applications with ten or more applications. Create a list of Names

names_to_replace = list(name_count[name_count <10].index)

# Replace in dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")

# Check to make sure binning was success
application_df['NAME'].value_counts()

Other                                      21022
PARENT BOOSTER USA INC                      1260
TOPS CLUB INC                                765
UNITED STATES BOWLING CONGRESS INC           700
WASHINGTON STATE UNIVERSITY                  492
                                           ...  
CASCADE 4-H FOUNDATION                        10
FREE & ACCEPTED MASONS OF WASHINGTON          10
NEW MEXICO GARDEN CLUBS INC                   10
NATIONAL ASSOCIATION OF HISPANIC NURSES       10
UNION OF CALIFORNIA STATE WORKERS             10
Name: NAME, Length: 223, dtype: int64

In [6]:
# Continue as before: Look at APPLICATION_TYPE value counts for binning
app_count = application_df['APPLICATION_TYPE'].value_counts()
print(app_count)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [7]:

# use the variable name `application_types_to_replace` with <1000
application_types_to_replace = list(app_count[app_count <1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was success
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced of <1000

classifications_to_replace = list(classification_count[classification_count <1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_df = pd.get_dummies(application_df)
numeric_df

STATUS   ASK_AMT  IS_SUCCESSFUL  \
0           1      5000              1   
1           1    108590              1   
2           1      5000              0   
3           1      6692              1   
4           1    142590              1   
...       ...       ...            ...   
34294       1      5000              0   
34295       1      5000              0   
34296       1      5000              0   
34297       1      5000              1   
34298       1  36500179              0   

       NAME_ACE MENTOR PROGRAM OF AMERICA INC  NAME_AIR FORCE ASSOCIATION  \
0                                           0                           0   
1                                           0                           0   
2                                           0                           0   
3                                           0                           0   
4                                           0                           0   
...                                       ...                         ...   
34294                                       0                           0   
34295                                       0                           0   
34296                                       0                           0   
34297                                       0                           0   
34298                                       0                           0   

       NAME_ALABAMA FEDERATION OF WOMENS CLUBS  \
0                                            0   
1                                            0   
2                                            0   
3                                            0   
4                                            0   
...                                        ...   
34294                                        0   
34295                                        0   
34296                                        0   
34297                                        0   
34298                                        0   

       NAME_ALABAMA TREASURE FOREST ASSOCIATION  NAME_ALPHA PHI SIGMA  \
0                                             0                     0   
1                                             0                     0   
2                                             0                     0   
3                                             0                     0   
4                                             0                     0   
...                                         ...                   ...   
34294                                         0                     0   
34295                                         0                     0   
34296                                         0                     0   
34297                                         0                     0   
34298                                         0                     0   

       NAME_ALPHA PHI SIGMA INC  NAME_ALTRUSA INTERNATIONAL FOUNDATION INC  \
0                             0                                          0   
1                             0                                          0   
2                             0                                          0   
3                             0                                          0   
4                             0                                          0   
...                         ...                                        ...   
34294                         0                                          0   
34295                         0                                          0   
34296                         0                                          0   
34297                         0                                          0   
34298                         0                                          0   

       ...  INCOME_AMT_1-9999  INCOME_AMT_10000-24999  \
0      ...                  0                       0   
1      ...                  1                   

In [11]:
# Split our preprocessed data into our features and target arrays
y = numeric_df["IS_SUCCESSFUL"].values

X = numeric_df.drop(["IS_SUCCESSFUL"], axis='columns').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  90
hidden_nodes_layer2 = 30
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 90)                23310     
                                                                 
 dense_1 (Dense)             (None, 10)                910       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 24341 (95.08 KB)
Trainable params: 24341 (95.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model (Adding in here the saving weights every 5 epochs - code thanks to umotto on : 
# https://stackoverflow.com/questions/51186330/save-model-weights-at-the-end-of-every-n-epochs), also 
# https://medium.com/@italojs/saving-your-weights-for-each-epoch-keras-callbacks-b494d9648202

mc = keras.callbacks.ModelCheckpoint('my_best_model.hdf5', 
                                     save_weights_only=True, period=5)


fit_model = nn.fit(X_train_scaled,y_train,epochs=75, callbacks=[mc])

Epoch 1/75
804/804 [==============================] - 2s 2ms/step - loss: 0.7105 - accuracy: 0.7532
Epoch 2/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5386 - accuracy: 0.7789
Epoch 3/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5353 - accuracy: 0.7822
Epoch 4/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5317 - accuracy: 0.7825
Epoch 5/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5269 - accuracy: 0.7816
Epoch 6/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5210 - accuracy: 0.7848
Epoch 7/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5151 - accuracy: 0.7863
Epoch 8/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5117 - accuracy: 0.7861
Epoch 9/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5062 - accuracy: 0.7866
Epoch 10/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5077 - accuracy: 0.7879

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5057 - accuracy: 0.7834 - 576ms/epoch - 2ms/step
Loss: 0.5057404637336731, Accuracy: 0.7834402322769165


In [29]:
# Define the model - Second attempt 
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  90
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 90)                23310     
                                                                 
 dense_29 (Dense)            (None, 30)                2730      
                                                                 
 dense_30 (Dense)            (None, 10)                310       
                                                                 
 dense_31 (Dense)            (None, 1)                 11        
                                                                 
Total params: 26361 (102.97 KB)
Trainable params: 26361 (102.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model -second attempts

mc = keras.callbacks.ModelCheckpoint('my_best_model.hdf5', 
                                     save_weights_only=True, period=5)


fit_model = nn.fit(X_train_scaled,y_train,epochs=75, callbacks=[mc])

Epoch 1/75
804/804 [==============================] - 3s 2ms/step - loss: 0.6715 - accuracy: 0.7601
Epoch 2/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5562 - accuracy: 0.7745
Epoch 3/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5386 - accuracy: 0.7784
Epoch 4/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5213 - accuracy: 0.7834
Epoch 5/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5215 - accuracy: 0.7849
Epoch 6/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5322 - accuracy: 0.7841
Epoch 7/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5390 - accuracy: 0.7842
Epoch 8/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5205 - accuracy: 0.7841
Epoch 9/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5099 - accuracy: 0.7839
Epoch 10/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5093 - accuracy: 0.7855

In [32]:
# Evaluate the model using the test data - Attempt 3 ( slight tweak to activation, but marginally less 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5063 - accuracy: 0.7812 - 456ms/epoch - 2ms/step
Loss: 0.5062602162361145, Accuracy: 0.7812244892120361


In [34]:
# Export our model to HDF5 file

nn.save("AlphabetSoupCharity_Optimization.hdf5")

# Optimising the code
After failing to get any significant optimisation through usual means, changing the neurons and layers, and changing the activations, I changed the level of binning, which also had little effect.
The final change that had the most successful change in optimisation was not dropping the "Name" column, and instead, keeping it and binning those names together with fewer than 10 values (applications). This increased the accuracy of the model above the requested 75%.
